In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# set theme
sns.set_theme(
    context="paper",
    style="whitegrid",
    rc={
        "figure.figsize": (7.0, 3.5),
        "savefig.dpi": 300,
        "figure.autolayout": True,
    },
)


fig_path = Path("../figures")

file = "scalability.parquet"

# Load Data

In [ ]:
data = pd.read_parquet(fig_path / file)
data

# Plot

In [ ]:
# Create the line plot
crosssections = pd.concat(
    [data[data["step"] == 24],
    data[data["step"] == 49],
    data[data["step"] == 99],
    data[data["step"] == 149],
    data[data["step"] == 199],]
)
crosssections["step"] += 1
crosssections["Time"] = crosssections["step"] * 0.1
crosssections = crosssections.groupby(["n_agents", "step"]).mean()

sns.relplot(
    data=crosssections,
    x="n_agents",
    y="coverage",
    # hue="step",
    style="Time",
    kind="line",
    errorbar="se",
    aspect=1.5,
)
plt.xlabel("$N$")
plt.xlim(0, 500)
plt.ylabel("$R$")
plt.ylim(0, 1)

# Save the plot
plt.savefig(fig_path / "coverage_transfer_final_scaled.png")
plt.savefig(fig_path / "coverage_transfer_final_scaled.pdf")
plt.show()